<a href="https://colab.research.google.com/github/Satan07/LGMVIP-DataScience/blob/main/BEGINEER_TASK_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))

Current working directory: /content
os.getcwd() returns an object of type: <class 'str'>


In [ ]:
cd songs

In [10]:
ls

members.csv   sample_submission.csv  songs.csv  train.csv
sample_data/  song_extra_info.csv    test.csv


In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb

In [16]:
print('Loading data...')
# data_path = "../songs/"
train = pd.read_csv('train.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                  'target' : np.uint8,
                                                  'song_id' : 'category'})
test = pd.read_csv('test.csv', dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                'source_screen_name' : 'category',
                                                'source_type' : 'category',
                                                'song_id' : 'category'})
songs = pd.read_csv('songs.csv',dtype={'genre_ids': 'category',
                                                  'language' : 'category',
                                                  'artist_name' : 'category',
                                                  'composer' : 'category',
                                                  'lyricist' : 'category',
                                                  'song_id' : 'category'})
members = pd.read_csv('members.csv',
                      dtype={'city' : 'category',
                             'bd' : np.uint8,
                             'gender' : 'category',
                             'registered_via' : 'category'}
                     ,parse_dates=["registration_init_time","expiration_date"])
songs_extra = pd.read_csv('song_extra_info.csv')

print('Data preprocessing...')

Loading data...
Data preprocessing...


In [17]:
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
# exepting some unimportanat features


# Convert date to number of days
members['membership_days'] = (members['expiration_date'] - members['registration_init_time']).dt.days.astype(int)

In [18]:
# categorize membership_days 
members['membership_days'] = members['membership_days']//200
members['membership_days'] = members['membership_days'].astype('category')

In [19]:
member_cols = ['msno','city','registered_via', 'registration_year', 'expiration_year', 'membership_days']

train = train.merge(members[member_cols], on='msno', how='left')
test = test.merge(members[member_cols], on='msno', how='left')

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2147645 entries, 0 to 2147644
Data columns (total 15 columns):
 #   Column              Dtype   
---  ------              -----   
 0   msno                object  
 1   song_id             object  
 2   source_system_tab   category
 3   source_screen_name  category
 4   source_type         category
 5   target              uint8   
 6   artist_name         category
 7   genre_ids           category
 8   song_length         float64 
 9   language            category
 10  city                category
 11  registered_via      category
 12  registration_year   int64   
 13  expiration_year     int64   
 14  membership_days     category
dtypes: category(9), float64(1), int64(2), object(2), uint8(1)
memory usage: 138.7+ MB


In [21]:
# song_extra (isrc) Setting and Merge

def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return int(isrc[5:7])//5
        else:
            return int(isrc[5:7])//5
    else:
        return np.nan
#categorize song_year per 5years

songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

In [22]:
train = train.merge(songs_extra, on = 'song_id', how = 'left')
test = test.merge(songs_extra, on = 'song_id', how = 'left')

In [23]:
train['genre_ids'] = train['genre_ids'].str.split('|').str[0]

In [24]:
temp_song_length = train['song_length']

In [25]:
train.drop('song_length', axis = 1, inplace = True)
test.drop('song_length',axis = 1 , inplace =True)

In [26]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,language,city,registered_via,registration_year,expiration_year,membership_days,song_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,52.0,1,7,2012,2017,10,3.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,52.0,13,9,2011,2017,11,19.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,52.0,13,9,2011,2017,11,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,-1.0,13,9,2011,2017,11,2.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,52.0,1,7,2012,2017,10,3.0


In [27]:
train.tail()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,language,city,registered_via,registration_year,expiration_year,membership_days,song_year
2147640,QaMS+JdfhMUtJwLZzf5TXo9SK0ylxxThD7CO58zAibA=,RxwDYPiGCMC5J0lQzmIDjbqcYg/Nh0gWsetF/Ojyzv8=,my library,Local playlist more,local-library,1,張心傑 (Mark Chang),465,3.0,4,3,2014,2017,6,2.0
2147641,QaMS+JdfhMUtJwLZzf5TXo9SK0ylxxThD7CO58zAibA=,SHV52/0W8kZcf3uJZF9+ZP3S5vu2RO4WNGhZ8qQHulU=,my library,Local playlist more,local-library,1,G.E.M.鄧紫棋,465,3.0,4,3,2014,2017,6,3.0
2147642,QaMS+JdfhMUtJwLZzf5TXo9SK0ylxxThD7CO58zAibA=,UPIWRmM9McD/HSS/ygwJEwN5nr10LgyIQ7RODjSLMec=,my library,Local playlist more,local-library,1,蔡健雅 (Tanya Chua),465,3.0,4,3,2014,2017,6,2.0
2147643,dU4RbzpIRRd/EkA9Xncpy9CglzDBZp7nKMfdnfr3Aj8=,DUEHe3LHGaK2nVA34Z7ho1U/FVIneTcRZFYI9zdZxDY=,explore,Online playlist more,online-playlist,0,Various Artists,921,3.0,22,9,2014,2017,6,3.0
2147644,dU4RbzpIRRd/EkA9Xncpy9CglzDBZp7nKMfdnfr3Aj8=,66NCw1qHAROy0PiUC1tPNmGYqRvBSfvGUsO+46BooDc=,explore,Online playlist more,online-playlist,1,Café Waiting Love O.S.T,458,3.0,22,9,2014,2017,6,2.0


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2147645 entries, 0 to 2147644
Data columns (total 15 columns):
 #   Column              Dtype   
---  ------              -----   
 0   msno                object  
 1   song_id             object  
 2   source_system_tab   category
 3   source_screen_name  category
 4   source_type         category
 5   target              uint8   
 6   artist_name         category
 7   genre_ids           object  
 8   language            category
 9   city                category
 10  registered_via      category
 11  registration_year   int64   
 12  expiration_year     int64   
 13  membership_days     category
 14  song_year           float64 
dtypes: category(8), float64(1), int64(2), object(3), uint8(1)
memory usage: 151.0+ MB


In [29]:
train.describe()

,target,registration_year,expiration_year,song_year
count,2.147645e+06,2.147645e+06,2.147645e+06,1.979540e+06
mean,6.367491e-01,2.012680e+03,2.017064e+03,2.944803e+00
std,4.809364e-01,2.981748e+00,4.029288e-01,3.627155e+00
min,0.000000e+00,2.004000e+03,2.004000e+03,0.000000e+00
25%,0.000000e+00,2.011000e+03,2.017000e+03,2.000000e+00
50%,1.000000e+00,2.013000e+03,2.017000e+03,2.000000e+00
75%,1.000000e+00,2.015000e+03,2.017000e+03,3.000000e+00
max,1.000000e+00,2.016000e+03,2.020000e+03,1.900000e+01


In [30]:
train.describe()

,target,registration_year,expiration_year,song_year
count,2.147645e+06,2.147645e+06,2.147645e+06,1.979540e+06
mean,6.367491e-01,2.012680e+03,2.017064e+03,2.944803e+00
std,4.809364e-01,2.981748e+00,4.029288e-01,3.627155e+00
min,0.000000e+00,2.004000e+03,2.004000e+03,0.000000e+00
25%,0.000000e+00,2.011000e+03,2.017000e+03,2.000000e+00
50%,1.000000e+00,2.013000e+03,2.017000e+03,2.000000e+00
75%,1.000000e+00,2.015000e+03,2.017000e+03,3.000000e+00
max,1.000000e+00,2.016000e+03,2.020000e+03,1.900000e+01


In [33]:
song_count = train.loc[:,["song_id","target"]]

# measure repeat count by played songs
song_count1 = song_count.groupby(["song_id"],as_index=False).sum().rename(columns={"target":"repeat_count"})

# count play count by songs
song_count2 = song_count.groupby(["song_id"],as_index=False).count().rename(columns = {"target":"play_count"})

In [34]:
song_repeat = song_count1.merge(song_count2,how="inner",on="song_id")
song_repeat["repeat_percentage"] = round((song_repeat['repeat_count']*100) / song_repeat['play_count'],1)
song_repeat['repeat_count'] = song_repeat['repeat_count'].astype('int')
song_repeat['repeat_percentage'] = song_repeat['repeat_percentage'].replace(100.0,np.nan)
#cuz most of 100.0 are played=1 repeated=1 values. I think it is not fair compare with other played a lot songs

In [35]:
train = train.merge(song_repeat,on="song_id",how="left")
test = test.merge(song_repeat,on="song_id",how="left")

In [36]:
# type cast
test['song_id'] = test['song_id'].astype('category')
test['repeat_count'] = test['repeat_count'].fillna(0)
test['repeat_count'] = test['repeat_count'].astype('int')
test['play_count'] = test['play_count'].fillna(0)
test['play_count'] = test['play_count'].astype('int')

In [37]:
artist_count = train.loc[:,["artist_name","target"]]

# measure repeat count by played songs
artist_count1 = artist_count.groupby(["artist_name"],as_index=False).sum().rename(columns={"target":"repeat_count_artist"})

# measure play count by songs
artist_count2 = artist_count.groupby(["artist_name"],as_index=False).count().rename(columns = {"target":"play_count_artist"})

artist_repeat = artist_count1.merge(artist_count2,how="inner",on="artist_name")

In [38]:
artist_repeat["repeat_percentage_artist"] = round((artist_repeat['repeat_count_artist']*100) / artist_repeat['play_count_artist'],1)
artist_repeat['repeat_count_artist'] = artist_repeat['repeat_count_artist'].fillna(0)
artist_repeat['repeat_count_artist'] = artist_repeat['repeat_count_artist'].astype('int')
artist_repeat['repeat_percentage_artist'] = artist_repeat['repeat_percentage_artist'].replace(100.0,np.nan)

In [39]:
#use only repeat_percentage_artist
del artist_repeat['repeat_count_artist']

In [40]:
#merge it with artist_name to train dataframe
train = train.merge(artist_repeat,on="artist_name",how="left")
test = test.merge(artist_repeat,on="artist_name",how="left")

In [41]:
del train['artist_name']
del test['artist_name']

In [42]:
# msno count
msno_count = train.loc[:,["msno","target"]]

# count repeat count by played songs
msno_count1 = msno_count.groupby(["msno"],as_index=False).sum().rename(columns={"target":"repeat_count_msno"})

# count play count by songs
msno_count2 = msno_count.groupby(["msno"],as_index=False).count().rename(columns = {"target":"play_count_msno"})

msno_repeat = msno_count1.merge(msno_count2,how="inner",on="msno")

In [43]:
msno_repeat["repeat_percentage_msno"] = round((msno_repeat['repeat_count_msno']*100) / msno_repeat['play_count_msno'],1)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].fillna(0)
msno_repeat['repeat_count_msno'] = msno_repeat['repeat_count_msno'].astype('int')
#msno_repeat['repeat_percentage_msno'] = msno_repeat['repeat_percentage_msno'].replace(100.0,np.nan)
# it can be meaningful so do not erase 100.0 

In [44]:
#merge it with msno to train dataframe
train = train.merge(msno_repeat,on="msno",how="left")
test = test.merge(msno_repeat,on="msno",how="left")

In [45]:
import gc
#del members, songs; gc.collect();

for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')

In [46]:
train['song_year'] = train['song_year'].astype('category')
test['song_year'] = test['song_year'].astype('category')

In [47]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,genre_ids,language,city,registered_via,registration_year,expiration_year,membership_days,song_year,repeat_count,play_count,repeat_percentage,play_count_artist,repeat_percentage_artist,repeat_count_msno,play_count_msno,repeat_percentage_msno
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,359,52.0,1,7,2012,2017,10,3.0,49,95,51.6,357.0,60.5,969,1544,62.8
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,1259,52.0,13,9,2011,2017,11,19.0,1,1,NaN,81098.0,64.7,257,306,84.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,1259,52.0,13,9,2011,2017,11,1.0,2,2,NaN,68.0,36.8,257,306,84.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,1019,-1.0,13,9,2011,2017,11,2.0,1,1,NaN,1.0,NaN,257,306,84.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,1011,52.0,1,7,2012,2017,10,3.0,50,119,42.0,123.0,41.5,969,1544,62.8


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2147645 entries, 0 to 2147644
Data columns (total 22 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   msno                      category
 1   song_id                   category
 2   source_system_tab         category
 3   source_screen_name        category
 4   source_type               category
 5   target                    uint8   
 6   genre_ids                 category
 7   language                  category
 8   city                      category
 9   registered_via            category
 10  registration_year         int64   
 11  expiration_year           int64   
 12  membership_days           category
 13  song_year                 category
 14  repeat_count              int64   
 15  play_count                int64   
 16  repeat_percentage         float64 
 17  play_count_artist         float64 
 18  repeat_percentage_artist  float64 
 19  repeat_count_msno         int64   
 20  pl

In [49]:
drop_list = ['repeat_count','repeat_percentage',
             'repeat_percentage_artist',
             'repeat_count_msno','repeat_percentage_msno'
            ]
train = train.drop(drop_list,axis=1)
test = test.drop(drop_list,axis=1)

In [50]:
test

,id,msno,song_id,source_system_tab,source_screen_name,source_type,genre_ids,language,city,registered_via,registration_year,expiration_year,membership_days,song_year,play_count,play_count_artist,play_count_msno
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,458,3.0,1,7,2016.0,2017.0,2,2.0,179,4484.0,20.0
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,465,3.0,1,7,2016.0,2017.0,2,2.0,2263,38728.0,20.0
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist,2022,17.0,1,4,2016.0,2016.0,0,2.0,1,85.0,9.0
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,465,52.0,3,9,2007.0,2017.0,17,0.0,13,225.0,NaN
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,873,-1.0,3,9,2007.0,2017.0,17,2.0,1,44.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2120579,2120579,5nsTrJgNxkjpLBmZtn1rIPnCE7yZgxB36vk6jN3sjXE=,0En85v+wzTEMe+Umaij0w9HKcR3qyhwUQACLmxx7WRU=,radio,Radio,radio,465,31.0,13,7,2015.0,2017.0,4,2.0,3,103.0,92.0
2120580,2120580,5nsTrJgNxkjpLBmZtn1rIPnCE7yZgxB36vk6jN3sjXE=,Tf93Scu6MgsOkB0qbqO6wiN/6dnzSG5Y5xx1SeNsYRQ=,radio,Radio,radio,465,31.0,13,7,2015.0,2017.0,4,1.0,13,868.0,92.0
2120581,2120581,5nsTrJgNxkjpLBmZtn1rIPnCE7yZgxB36vk6jN3sjXE=,J9h4EwleeHLvdiAAcfwcAHAL6YO5I8rg9kz8zIwVmo4=,radio,Radio,radio,465,31.0,13,7,2015.0,2017.0,4,NaN,2,77.0,92.0
2120582,2120582,E667T1pytMqqKlT953tWtkE2Lb7RVXVI+0g62Z3tLnY=,l2MCI5crBPTjD4+oBKQsh5wHThW01T6bcoxLij2M9Pg=,my library,Local playlist more,local-library,465,3.0,12,7,2011.0,2017.0,10,1.0,11,12661.0,4.0


In [51]:
test['play_count_msno'] = test['play_count_msno'].fillna(0)
test['play_count_msno'] = test['play_count_msno'].astype('int')


train['play_count_artist'] = train['play_count_artist'].fillna(0)
test['play_count_artist'] = test['play_count_artist'].fillna(0)
train['play_count_artist'] = train['play_count_artist'].astype('int')
test['play_count_artist'] = test['play_count_artist'].astype('int')

In [ ]:
from sklearn.model_selection import KFold
# Create a Cross Validation with 3 splits
kf = KFold(n_splits=3)

predictions = np.zeros(shape=[len(test)])

# For each KFold
for train_indices ,validate_indices in kf.split(train) : 
    train_data = lgb.Dataset(train.drop(['target'],axis=1).loc[train_indices,:],label=train.loc[train_indices,'target'])
    val_data = lgb.Dataset(train.drop(['target'],axis=1).loc[validate_indices,:],label=train.loc[validate_indices,'target'])

    params = {
            'objective': 'binary',
            'boosting': 'gbdt',
            'learning_rate': 0.2 ,
            'verbose': 0,
            'num_leaves': 2**8,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 1,
            'feature_fraction': 0.9,
            'feature_fraction_seed': 1,
            'max_bin': 256,
            'num_rounds': 80,
            'metric' : 'auc'
        }    
    # Train the model    
    lgbm_model = lgb.train(params, train_data, 100, valid_sets=[val_data])
    predictions += lgbm_model.predict(test.drop(['id'],axis=1))
    del lgbm_model
    # We get the ammount of predictions from the prediction list, by dividing the predictions by the number of Kfolds.
predictions = predictions/3

# INPUT_DATA_PATH = '../input/'

# Read the sample_submission CSV
submission = pd.read_csv('sample_submission.csv')
# Set the target to our predictions
submission.target=predictions
# Save the submission file
submission.to_csv('submission.csv',index=False)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's auc: 0.759955
[2]	valid_0's auc: 0.768187
[3]	valid_0's auc: 0.77288
[4]	valid_0's auc: 0.77631
[5]	valid_0's auc: 0.78451
[6]	valid_0's auc: 0.787645
[7]	valid_0's auc: 0.790875
[8]	valid_0's auc: 0.794427
[9]	valid_0's auc: 0.795221
[10]	valid_0's auc: 0.796432
[11]	valid_0's auc: 0.797094
[12]	valid_0's auc: 0.797435
[13]	valid_0's auc: 0.798936
[14]	valid_0's auc: 0.800168
[15]	valid_0's auc: 0.800673
[16]	valid_0's auc: 0.801222
[17]	valid_0's auc: 0.801729
[18]	valid_0's auc: 0.801833
[19]	valid_0's auc: 0.802365
[20]	valid_0's auc: 0.802588
[21]	valid_0's auc: 0.802853
[22]	valid_0's auc: 0.80297
[23]	valid_0's auc: 0.802998
[24]	valid_0's auc: 0.802921
[25]	valid_0's auc: 0.802961
[26]	valid_0's auc: 0.80303
[27]	valid_0's auc: 0.80318
[28]	valid_0's auc: 0.803052
[29]	valid_0's auc: 0.802988
[30]	valid_0's auc: 0.802863
[31]	valid_0's auc: 0.802924
[32]	valid_0's auc: 0.802852
[33]	valid_0's auc: 0.802926
[34]	valid_0's auc: 0.802665
[35]	valid_0's auc: 0.802569
